note: Added comparison of 2019 data and 2020 data. As a final push to the LB score, it seems worthwhile to add an inference model with reference to the heatmap.

## I hope this prob-list is helpful for you!

It is a derivative of [this notebook](https://www.kaggle.com/marutama/optuna-tuning-resnext50-blending-inference).


Than kyou for great previous work. please, upvote them!

* [Cassava Leaf Disease - Exploratory Data Analysis](https://www.kaggle.com/ihelon/cassava-leaf-disease-exploratory-data-analysis)
* [Ensemble: Resnext50_32x4d + Efficientnet = 0.903](https://www.kaggle.com/japandata509/ensemble-resnext50-32x4d-efficientnet-0-903)
* [[No TTA] Cassava Resnext50_32x4d Inference lb0.903](https://www.kaggle.com/piantic/no-tta-cassava-resnext50-32x4d-inference-lb0-903/output)
* [Clean_Inference_Kernel_8xTTA_LB902](https://www.kaggle.com/underwearfitting/clean-inference-kernel-8xtta-lb902/data)
* [Cassava-ensemble-(efnetb3-resnet50)](https://www.kaggle.com/shubham108/cassava-ensemble-efnetb3-resnet50)

# EDA of Resnext and Efficientnet inference

In [ ]:
import os

import scipy as sp
from scipy.special import softmax
import numpy as np
import pandas as pd
import json

from sklearn.metrics import accuracy_score

from matplotlib import pyplot as plt
import seaborn as sns

import cv2

In [ ]:
BASE_DIR = "../input/cassava-leaf-disease-classification/"
with open(os.path.join(BASE_DIR, "label_num_to_disease_map.json")) as file:
    j = json.loads(file.read())

map_classes = {int(k) : v for k, v in j.items()}

In [ ]:
print(json.dumps(j, indent=4))
print(j["1"]) # index is character string

In [ ]:
map_classes[0] = 'CBB'
map_classes[1] = 'CBSD'
map_classes[2] = 'CGM'
map_classes[3] = 'CMD'

In [ ]:
print(json.dumps(map_classes, indent=4))
print(map_classes[4]) # index is integer

In [ ]:
# You can also use pprint to display the json object. This is generally useful.
from pprint import pprint
print('Index is character string.')
pprint(j)
print('Index is integer.')
pprint(map_classes)

# Read all-rnxt-effn.csv
This Dataframe is inference results of Resnext504d and Effcientnet-b4.

* image_id : Image filename of merged.csv including train.csv
* correct : label of merged.csv including train.csv
* Rnxt : Resnext inference labels
* Effn : Efficientnet inference labels
* 50%+50% : lResnext and Effcientnet ensemble labels
* r0_0 : Probability of Resnext model 0, category 0 
* r0_1 : Probability of Resnext model 0, category 1 
* ...
* e0_0 : Probability of Efficientnet model 0, category 0 
* e0_1 : Probability of Efficientnet model 0, category 1 
* ...

In [ ]:
a=pd.read_csv('../input/all-rnxt-effn/all-rnxt-effn.csv')
a

# Calculate confidence

Calculate confidence of '50%+50%'

In [ ]:
a['r_0']=a['r0_0']+a['r1_0']+a['r2_0']+a['r3_0']+a['r4_0']
a['r_1']=a['r0_1']+a['r1_1']+a['r2_1']+a['r3_1']+a['r4_1']
a['r_2']=a['r0_2']+a['r1_2']+a['r2_2']+a['r3_2']+a['r4_2']
a['r_3']=a['r0_3']+a['r1_3']+a['r2_3']+a['r3_3']+a['r4_3']
a['r_4']=a['r0_4']+a['r1_4']+a['r2_4']+a['r3_4']+a['r4_4']

a['e_0']=a['e0_0']+a['e1_0']+a['e2_0']+a['e3_0']+a['e4_0']
a['e_1']=a['e0_1']+a['e1_1']+a['e2_1']+a['e3_1']+a['e4_1']
a['e_2']=a['e0_2']+a['e1_2']+a['e2_2']+a['e3_2']+a['e4_2']
a['e_3']=a['e0_3']+a['e1_3']+a['e2_3']+a['e3_3']+a['e4_3']
a['e_4']=a['e0_4']+a['e1_4']+a['e2_4']+a['e3_4']+a['e4_4']

a['re_0']=a['r_0']+a['e_0']
a['re_1']=a['r_1']+a['e_1']
a['re_2']=a['r_2']+a['e_2']
a['re_3']=a['r_3']+a['e_3']
a['re_4']=a['r_4']+a['e_4']

a['confidence']=a[['re_0', 're_1', 're_2', 're_3', 're_4']].max(1)/10

In [ ]:
b=a[['image_id', 'correct', 'Rnxt', 'Effn', '50%+50%', 'confidence']]
b

In [ ]:
## Boundary between 2019 and 2020 data

In [ ]:
b2020 = b[:21396]
b2020

In [ ]:
b2019 = b[21396:]
b2019

# Plot of confidence

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(b2020['confidence'])
plt.plot(b2019['confidence'])

Confidence seems to be a bit high in the second half of 2019 data

In [ ]:
b['confidence'].describe()

In [ ]:
sns.displot(b['confidence'], height=5, aspect=2)
#sns.distplot(b['confidence']) # if your seaborn version is old

In [ ]:
b2020['confidence'].describe()

In [ ]:
sns.displot(b2020['confidence'], height=5, aspect=2)

In [ ]:
b2019['confidence'].describe()

In [ ]:
sns.displot(b2019['confidence'], height=5, aspect=2)

There seems to be no big difference in confidence distribution between 2020 and 2019.

In [ ]:
print('Merged acc', accuracy_score(b['correct'], b['50%+50%']))
print('2020   acc', accuracy_score(b2020['correct'], b2020['50%+50%']))
print('2019   acc', accuracy_score(b2019['correct'], b2019['50%+50%']))

There is a difference in accuracy between 2020 and 2019, but it is about 20%.

In [ ]:
plt.figure(figsize=(12, 4))
fig=sns.countplot(y='50%+50%', hue='correct', data=b)

In [ ]:
plt.figure(figsize=(12, 4))
fig=sns.countplot(y='50%+50%', hue='correct', data=b)
fig.set(xlim=(0,350))

In [ ]:
plt.figure(figsize=(12, 4))
fig=sns.countplot(y='50%+50%', hue='correct', data=b2020)
fig.set(xlim=(0,350))

In [ ]:
plt.figure(figsize=(12, 4))
fig=sns.countplot(y='50%+50%', hue='correct', data=b2019)
fig.set(xlim=(0,350))

In [ ]:
# Create a pivot table with the number of occurrences
m=b.pivot_table(index='correct', columns='50%+50%', values='image_id', aggfunc='count')
# Clip maximums for heatmap
m=m.clip(upper=int(278*1.2)) # 1.2 times the largest number other than the diagonal
plt.figure(figsize=(15, 5)) 
sns.heatmap(m, cmap='rainbow', annot=True, fmt='g')

In [ ]:
# Create a pivot table with the number of occurrences
m=b2020.pivot_table(index='correct', columns='50%+50%', values='image_id', aggfunc='count')
# Clip maximums for heatmap
m=m.clip(upper=int(220*1.2)) # 1.2 times the largest number other than the diagonal
plt.figure(figsize=(15, 5)) 
sns.heatmap(m, cmap='rainbow', annot=True, fmt='g')

In [ ]:
# Create a pivot table with the number of occurrences
m=b2019.pivot_table(index='correct', columns='50%+50%', values='image_id', aggfunc='count')
# Clip maximums for heatmap
m=m.clip(upper=int(58*1.2)) # 1.2 times the largest number other than the diagonal
plt.figure(figsize=(15, 5)) 
sns.heatmap(m, cmap='rainbow', annot=True, fmt='g')

ヒートマップを参考に別モデルを導入することは有効そうです。ただ対応パターンは2019と2020で違いそうです。

It seems effective to introduce another model with reference to the heat map. However, the correspondence pattern seems to be different between 2019 and 2020.

2020データでは下記のケースが比較的多いようだ:
* correct:4が間違えて50%+50%:0に予測されるケース
* correct:4が間違えて50%+50%:3に予測されるケース
* correct:2が間違えて50%+50%:3に予測されるケース
* correct:1が間違えて50%+50%:4に予測されるケース
* correct:0が間違えて50%+50%:4に予測されるケース


The following cases seem to be relatively common in the 2020 data:
* correct: 4 is mistakenly predicted to be 50%+50%: 0
* correct: 4 is mistakenly predicted to be 50%+50%: 3
* correct: 2 is mistakenly predicted to be 50%+50%: 3
* correct: 1 is mistakenly predicted to be 50%+50%: 4
* correct: 0 is mistakenly predicted to be 50%+50%: 4

# Show images

In [ ]:
def visualize_batch(df):
    
    image_ids = df["image_id"].values
    corrects = df['correct'].values
    labels = df["50%+50%"].values
    confidences = df['confidence'].values
    
    plt.figure(figsize=(16, 12))
    
    for ind, (image_id, correct, label, conf) in enumerate(zip(image_ids, corrects, labels, confidences)):
        plt.subplot(3, 3, ind + 1)
        image = cv2.imread(os.path.join('../input/cassava-leaf-disease-merged/train', image_id))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        plt.imshow(image)
        plt.title(f"correct:{correct}({map_classes[correct]}), 50%+50%:{label}({map_classes[label]}),\n{image_id}, conf={conf:.4f}", fontsize=12)
        plt.axis("off")
    
    plt.show()

## Display 9 images with low confidence value

In [ ]:
s=b.sort_values('confidence', ascending=True)[:9]
visualize_batch(s)

Images with different aspect ratios are 2019 data.

## Display 9 images with high confidence value

In [ ]:
s=b.sort_values('confidence', ascending=False)[:9]
visualize_batch(s)

# Correct label:0 Cassava Bacterial Blight (CBB)

Angular necrotic spotting of the leaves—often with a chlorotic ring encircling the spots.

葉の角のある壊死性の斑点—多くの場合、斑点を取り囲むクロロティックリングを伴う。

## High confidence

In [ ]:
s=b[b['correct']==0].sort_values('confidence', ascending=False)[:9]
visualize_batch(s)

## Low confidence

In [ ]:
s=b[b['correct']==0].sort_values('confidence', ascending=True)[:9]
visualize_batch(s)

# Correct label:1 Cassava Brown Streak Disease (CBSD)

Severe chlorosis and necrosis on infected leaves, giving them a yellowish, mottled appearance.

感染した葉の重度の白化と壊死により、黄色がかったまだらの外観になります。

## High confidence

In [ ]:
s=b[b['correct']==1].sort_values('confidence', ascending=False)[:9]
visualize_batch(s)

Hmmm, only the cross section of the root ...

In [ ]:
s=b[b['correct']==1].sort_values('confidence', ascending=False)[100:109]
visualize_batch(s)

## Low confidence

In [ ]:
s=b[b['correct']==1].sort_values('confidence', ascending=True)[:9]
visualize_batch(s)

# Correct label:2 Cassava Green Mottle (CGM)

Young leaves are puckered with faint to distinct yellow spots, green patterns (mosaics), and twisted margins.

若い葉は、かすかにはっきりとした黄色の斑点、緑色のパターン（モザイク）、およびねじれた縁でしわが寄っています。

## High confidence

In [ ]:
s=b[b['correct']==2].sort_values('confidence', ascending=False)[:9]
visualize_batch(s)

## Low confidence

In [ ]:
s=b[b['correct']==2].sort_values('confidence', ascending=True)[:9]
visualize_batch(s)

# Correct label:3 Cassava Mosaic Disease (CMD)

These symptoms include chlorotic mosaic of the leaves, leaf distortion, and stunted growth. Leaf stalks have a characteristic S-shape.

植物全体に病徴が発現する。病徴には葉のモザイク状の白化、葉の変形、生育の阻害が含まれる。

## High confidence

In [ ]:
s=b[b['correct']==3].sort_values('confidence', ascending=False)[:9]
visualize_batch(s)

## Low confidence

In [ ]:
s=b[b['correct']==3].sort_values('confidence', ascending=True)[:9]
visualize_batch(s)

# Correct label:4 Healthy

## High confidence

In [ ]:
s=b[b['correct']==4].sort_values('confidence', ascending=False)[:9]
visualize_batch(s)

## Low confidence

In [ ]:
s=b[b['correct']==4].sort_values('confidence', ascending=True)[:9]
visualize_batch(s)

In particular, there are clearly many images of illness on the Healthy correct label of the 2020 data. Label noise may be high in the 2020 data.
2020 healthy low confidence : about 0.24 - 0.34
2019 healthy low confidence : about 0.40 - 0.49

In [ ]:
s=b2019[b2019['correct']==4].sort_values('confidence', ascending=True)[:9]
visualize_batch(s)

# Displays images with the same label of correct, Rnxt, Effn and 50% + 50%

Images can be displayed in various patterns in this way.

In [ ]:
t=b[(b['correct']==b['Rnxt'])&(b['correct']==b['Effn'])&(b['correct']==b['50%+50%'])]
#t=b[(b['correct']==3) & (b['Rnxt']==3) & (b['Effn']==3) & (b['50%+50%']==3)]
s=t.sort_values('confidence', ascending=True)[:9]
visualize_batch(s)

In [ ]:
sns.displot(t['confidence'],height=5, aspect=2)

Other images

In [ ]:
t=b[~((b['correct']==b['Rnxt'])&(b['correct']==b['Effn'])&(b['correct']==b['50%+50%']))]
#t=b[(b['correct']==3) & (b['Rnxt']==3) & (b['Effn']==3) & (b['50%+50%']==3)]
s=t.sort_values('confidence', ascending=True)[:9]
visualize_batch(s)

In [ ]:
sns.displot(t['confidence'],height=5, aspect=2)

confidence が高いのに correctと異なる予測になっているのは label noiseが多いからでしょう。

The reason why the prediction is different from correct even though the confidence is high is probably because there is a lot of label noise.

# Let's check our assumptions

* correct: 4 is mistakenly predicted to be 50%+50%: 0
* correct: 4 is mistakenly predicted to be 50%+50%: 3
* correct: 2 is mistakenly predicted to be 50%+50%: 3
* correct: 1 is mistakenly predicted to be 50%+50%: 4
* correct: 0 is mistakenly predicted to be 50%+50%: 4

Let's look at only the first two here.


## correct: 4 is mistakenly predicted to be 50%+50%: 0

In [ ]:
b.groupby('50%+50%').mean()['confidence']

In [ ]:
s=b[(b['correct']==4)&(b['50%+50%']==0)].sort_values('confidence', ascending=True)[:9]
visualize_batch(s)

## correct: 4 is mistakenly predicted to be 50%+50%: 3

In [ ]:
s=b[(b['correct']==4)&(b['50%+50%']==3)].sort_values('confidence', ascending=True)[:9]
visualize_batch(s)

## I hope it helps you ...

In [ ]:
# If you want to execute the following code, comment out this Assert and turn it on with GPU Accelerator.
# It takes 4 hours.
assert(False)

# How to make 'all-rnxt-effn.csv'

## Import Library

In [ ]:
# ====================================================
# Library
# ====================================================
import sys
import os
import math
import time
import random
import shutil
import albumentations
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
from scipy.special import softmax
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

In [ ]:
# ====================================================
# Library for Pytorch and GPU
# ====================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensorV2

sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
import timm

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

## Directory settings

In [ ]:
# ====================================================
# Directory settings for Resnext
# ====================================================
OUTPUT_DIR = './'
MODEL_DIR = '../input/cassava-resnext50-32x4d-weights/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    
TRAIN_PATH = '../input/cassava-leaf-disease-classification/train_images'
#TEST_PATH = '../input/cassava-leaf-disease-classification/test_images'
MERGED_PATH = '../input/cassava-leaf-disease-merged/train'
TEST_PATH = MERGED_PATH
MERGED_CSV = '../input/cassava-leaf-disease-merged/merged.csv'
#SUBM_CSV = '../input/cassava-leaf-disease-classification/sample_submission.csv   '

## Configuration setting

In [ ]:
#BATCH_SIZE ... ResNext inference time(min) : in case of 26337(MERGED num) 512x512 img, GCP Tesla T4
#       T4     ,  P100(Kaggle)
#01 ... 86(min)
#02 ... 64(min)
#04 ... 53(min)
#08 ... 51(min)
#16 ... 47(min),  35(min)
#32 ... 46(min),  34(min)
#64 ... 45(min)
BATCH_SIZE = 32
total_file_nums = len(os.listdir(TEST_PATH))
d = total_file_nums // BATCH_SIZE
max_file_nums = d * BATCH_SIZE
print(f'Adjust the number of input files by batch size: {total_file_nums}->{max_file_nums}')

In [ ]:
%%time
inp_imgs= []
for dirname, _, filenames in os.walk(TEST_PATH):
    for filename in filenames[:max_file_nums]:
    #for filename in filenames[:BATCH_SIZE*16]: # デバッグ用入力画像数絞れる
        #print(os.path.join(dirname, filename))
        #print(filename)
        inp_imgs.append(filename)
inp_imgs.sort()
print(len(inp_imgs))

In [ ]:
# ====================================================
# CFG for ResNext
# ====================================================
class CFG:
    debug=False
    num_workers=0  # original is 8
    model_name='resnext50_32x4d'
    size=512
    batch_size=BATCH_SIZE # original is 32
    seed=2020
    target_size=5
    target_col='label'
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    inference=True

In [ ]:
#for efficientnet
EFF_BATCH_SIZE = 1 # Important: Increasing the batch size will decrease the Effn score
# EFF_BATCH_SIZE=1 ... Effn inference is about 60min@T4(50min@P100) x 5 models
image_size = 512
enet_type = ['tf_efficientnet_b4_ns'] * 5
model_path = ['../input/moa-b4-baseline/baseline_cld_fold0_epoch8_tf_efficientnet_b4_ns_512.pth', 
              '../input/moa-b4-baseline/baseline_cld_fold1_epoch9_tf_efficientnet_b4_ns_512.pth', 
              '../input/moa-b4-baseline/baseline_cld_fold2_epoch9_tf_efficientnet_b4_ns_512.pth',
              '../input/moa-b4-baseline/baseline_cld_fold3_epoch5_tf_efficientnet_b4_ns_512.pth',
              '../input/moa-b4-baseline/baseline_cld_fold4_epoch11_tf_efficientnet_b4_ns_512.pth']

In [ ]:
#Transform for efficientnet
transforms_valid = albumentations.Compose([
    albumentations.Resize(800, 600), # For 2019 data, it is smaller than 512x512
    albumentations.CenterCrop(image_size, image_size, p=1),
    albumentations.Resize(image_size, image_size),
    albumentations.Normalize()
])

## Utils

In [ ]:
# ====================================================
# Utils for Resnext
# ====================================================
def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)


@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')


def init_logger(log_file=OUTPUT_DIR+'inference.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

#LOGGER = init_logger()

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

## Data Loading

In [ ]:
merged = pd.read_csv(MERGED_CSV)
merged = merged.set_index('image_id')
#merged

In [ ]:
len(inp_imgs)
# 21397 TRAIN num
# 26337 MERGED num

In [ ]:
tmp = merged.loc[inp_imgs]
test = tmp.rename(columns={'label':'correct'}).drop('source', axis=1)

In [ ]:
test = test.reset_index()
#test.head()

In [ ]:
# filepath is for Effcientnet dataloader
#test = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
test['filepath'] = test.image_id.apply(lambda x: os.path.join(MERGED_PATH, f'{x}'))
test.head()

## Dataset

In [ ]:
# ====================================================
# Dataset for Resnext
# ====================================================
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

In [ ]:
# ====================================================
# Dataset for efficientnet
# ====================================================
class CLDDataset(Dataset):
    def __init__(self, df, mode, transform=None):
        self.df = df.reset_index(drop=True)
        self.mode = mode
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.loc[index]
        image = cv2.imread(row.filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            res = self.transform(image=image)
            image = res['image']
        
        image = image.astype(np.float32)
        image = image.transpose(2,0,1)
        if self.mode == 'test':
            return torch.tensor(image).float()
        else:
            return torch.tensor(image).float(), torch.tensor(row.label).float()

In [ ]:
#for efficientnet
test_dataset_efficient = CLDDataset(test, 'test', transform=transforms_valid)
test_loader_efficient = torch.utils.data.DataLoader(test_dataset_efficient, batch_size=EFF_BATCH_SIZE, shuffle=False,  num_workers=0) # original num_workders=4

In [ ]:
# ====================================================
# Transforms for Resnext
# ====================================================
def get_transforms(*, data):
    if data == 'valid':
        return A.Compose([
            A.Resize(CFG.size, CFG.size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

## MODELS

In [ ]:
# ====================================================
# ResNext Model
# ====================================================
class CustomResNext(nn.Module):
    def __init__(self, model_name='resnext50_32x4d', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, CFG.target_size)

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
# ====================================================
# EfficientNet Model
# ====================================================
class enet_v2(nn.Module):

    def __init__(self, backbone, out_dim, pretrained=False):
        super(enet_v2, self).__init__()
        self.enet = timm.create_model(backbone, pretrained=pretrained)
        in_ch = self.enet.classifier.in_features
        self.myfc = nn.Linear(in_ch, out_dim)
        self.enet.classifier = nn.Identity()

    def forward(self, x):
        x = self.enet(x)
        x = self.myfc(x)
        return x

## Helper functions

In [ ]:
# ====================================================
# Helper functions for Resnext
# ====================================================
def load_state(model_path):
    model = CustomResNext(CFG.model_name, pretrained=False)
    try:  # single GPU model_file
        model.load_state_dict(torch.load(model_path)['model'], strict=True)
        state_dict = torch.load(model_path)['model']
    except:  # multi GPU model_file
        state_dict = torch.load(model_path)['model']
        state_dict = {k[7:] if k.startswith('module.') else k: state_dict[k] for k in state_dict.keys()}

    return state_dict

def inference(model, states, test_loader, device):
    model.to(device)
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader), desc='[Rnxt All]')
    probs = []
    for i, (images) in tk0:
        images = images.to(device)
        avg_preds = []
        for state in states:
            #model.load_state_dict(state['model'])
            model.load_state_dict(state)
            model.eval()
            with torch.no_grad():
                y_preds = model(images)
            avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
        #avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs

#tta_inference for ResNextを作るべきか？

In [ ]:
# ====================================================
# Helper functions for efficientnet
# ====================================================
def inference_func(test_loader):
    model.eval()
    bar = tqdm(test_loader, desc='[Effn]')

    LOGITS = []
    PREDS = []
    
    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            logits = model(x)
            LOGITS.append(logits.cpu())
            PREDS += [torch.softmax(logits, 1).detach().cpu()]
        PREDS = torch.cat(PREDS).cpu().numpy()
        LOGITS = torch.cat(LOGITS).cpu().numpy()
    return PREDS

def tta_inference_func(test_loader):
    model.eval()
    bar = tqdm(test_loader, desc='[Effn TTA]')
    PREDS = []
    LOGITS = []

    with torch.no_grad():
        for batch_idx, images in enumerate(bar):
            x = images.to(device)
            x = torch.stack([x,x.flip(-1),x.flip(-2),x.flip(-1,-2),
            x.transpose(-1,-2),x.transpose(-1,-2).flip(-1),
            x.transpose(-1,-2).flip(-2),x.transpose(-1,-2).flip(-1,-2)],0)
            x = x.view(-1, 3, image_size, image_size)
            logits = model(x)
            logits = logits.view(EFF_BATCH_SIZE, 8, -1).mean(1)
            PREDS += [torch.softmax(logits, 1).detach().cpu()]
            LOGITS.append(logits.cpu())

        PREDS = torch.cat(PREDS).cpu().numpy()
        
    return PREDS

## Inference ResNext

In [ ]:
%%time
model = CustomResNext(CFG.model_name, pretrained=False)
#model = enet_v2(enet_type[i], out_dim=5)
states = [load_state(MODEL_DIR+f'{CFG.model_name}_fold{fold}.pth') for fold in CFG.trn_fold]
test_dataset = TestDataset(test, transform=get_transforms(data='valid')) # NO TTA
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True)
probs_rnxt = inference(model, states, test_loader, device)

In [ ]:
# Split by model
MODELS = 5
r = [0]*MODELS
for i in range(len(probs_rnxt)//MODELS):
    for m in range(MODELS):
        if i==0:
            r[m]=probs_rnxt[m]
        else:
            r[m] = np.concatenate([r[m], probs_rnxt[i*5+m]])

## Inference Effcientnet

In [ ]:
%%time
e = []
for i in range(len(enet_type)):
    model = enet_v2(enet_type[i], out_dim=5)
    model = model.to(device)
    model.load_state_dict(torch.load(model_path[i]))
    e += [tta_inference_func(test_loader_efficient)]

In [ ]:
print(np.array(r).shape, np.array(e).shape)

 # Save Results 

In [ ]:
test = test.drop('filepath', axis=1)

In [ ]:
# Score : RexNext
pred = np.mean(r, axis=0)
test['Rnxt'] = softmax(pred).argmax(1)
# acc_scoreで検算チェック
acc_score = accuracy_score(test['correct'], test['Rnxt'])
print(acc_score)
# 0.909753703790251 ... TRAIN score
# 0.8880282492311197 ... MERGED score

In [ ]:
# Score : Effcientnet
pred = np.mean(e, axis=0)
test['Effn'] = softmax(pred).argmax(1)
# acc_scoreで検算チェック
acc_score = accuracy_score(test['correct'], test['Effn'])
print(acc_score)
# 0.909753703790251 ... TRAIN score
# 0.8880282492311197 ... MERGED score

In [ ]:
pred = 0.50*np.mean(r, axis=0) + 0.50*np.mean(e, axis=0) # lb 0.903
test['50%+50%'] = softmax(pred).argmax(1)
#test[['image_id', '50%+50%']].to_csv(OUTPUT_DIR+'submission.csv', index=False) # must change 50%+50% -> label
# acc_scoreで検算チェック
acc_score = accuracy_score(test['correct'], test['50%+50%'])
print(acc_score)
# 0.909753703790251 ... TRAIN score
# 0.8880282492311197 ... MERGED score
test.head()

## Merge

In [ ]:
col = ['r0_0', 'r0_1', 'r0_2', 'r0_3', 'r0_4']
dr0 = pd.DataFrame(data=r[0], columns=col, dtype='float32')
col = ['r1_0', 'r1_1', 'r1_2', 'r1_3', 'r1_4']
dr1 = pd.DataFrame(data=r[1], columns=col, dtype='float32')
col = ['r2_0', 'r2_1', 'r2_2', 'r2_3', 'r2_4']
dr2 = pd.DataFrame(data=r[2], columns=col, dtype='float32')
col = ['r3_0', 'r3_1', 'r3_2', 'r3_3', 'r3_4']
dr3 = pd.DataFrame(data=r[3], columns=col, dtype='float32')
col = ['r4_0', 'r4_1', 'r4_2', 'r4_3', 'r4_4']
dr4 = pd.DataFrame(data=r[4], columns=col, dtype='float32')

In [ ]:
col = ['e0_0', 'e0_1', 'e0_2', 'e0_3', 'e0_4']
de0 = pd.DataFrame(data=e[0], columns=col, dtype='float32')
col = ['e1_0', 'e1_1', 'e1_2', 'e1_3', 'e1_4']
de1 = pd.DataFrame(data=e[1], columns=col, dtype='float32')
col = ['e2_0', 'e2_1', 'e2_2', 'e2_3', 'e2_4']
de2 = pd.DataFrame(data=e[2], columns=col, dtype='float32')
col = ['e3_0', 'e3_1', 'e3_2', 'e3_3', 'e3_4']
de3 = pd.DataFrame(data=e[3], columns=col, dtype='float32')
col = ['e4_0', 'e4_1', 'e4_2', 'e4_3', 'e4_4']
de4 = pd.DataFrame(data=e[4], columns=col, dtype='float32')

In [ ]:
all_df = pd.merge(test,   dr0, left_index=True, right_index=True)
all_df = pd.merge(all_df, dr1, left_index=True, right_index=True)
all_df = pd.merge(all_df, dr2, left_index=True, right_index=True)
all_df = pd.merge(all_df, dr3, left_index=True, right_index=True)
all_df = pd.merge(all_df, dr4, left_index=True, right_index=True)

all_df = pd.merge(all_df, de0, left_index=True, right_index=True)
all_df = pd.merge(all_df, de1, left_index=True, right_index=True)
all_df = pd.merge(all_df, de2, left_index=True, right_index=True)
all_df = pd.merge(all_df, de3, left_index=True, right_index=True)
all_df = pd.merge(all_df, de4, left_index=True, right_index=True)

all_df.to_csv("all-rnxt-effn.csv", index=False)

In [ ]:
all_df.head()

In [ ]:
test[test['Rnxt']!=test['Effn']]